<a href="https://colab.research.google.com/github/C4Phoenix/simpelKeggleIntroToML/blob/master/fraudDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# preperation
## install needed libraries

In [0]:
#!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
#!pip install --upgrade wandb
#!wandb login f244ffe13b0872010de2092f7a8fe61186506c10
!pip install https://github.com/tqdm/tqdm/archive/master.zip

     | 890kB 3.3MB/s
  Created wheel for tqdm: filename=tqdm-4.35.0-py2.py3-none-any.whl size=50827 sha256=4dd8799dbea06efc5c898fdcaa5a2180cc4d6d0134d3a4902c9abd3d2220b0dc
  Stored in directory: /tmp/pip-ephem-wheel-cache-t0_ic94_/wheels/99/27/06/552c52d31c15337736a81cdd2d0f2a504c36ac1fe266a724f2
Successfully built tqdm
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


## mount google drive for importing data

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## import some libraries

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from tensorflow import keras

## load data

In [0]:
missing_values = ["NaN"]
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Datasets/fraud_detection/train_transaction.csv', na_values=missing_values)
df_identity = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Datasets/fraud_detection/train_identity.csv', na_values=missing_values)
df = pd.merge(df, df_identity, on='TransactionID', how='left')
del df_identity
df.shape

(590540, 434)

# Pre prosessing
## understanding the data
get a feel for what the collums are

In [0]:
print( df.columns.tolist())

['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt', 'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'dist1', 'dist2', 'P_emaildomain', 'R_emaildomain', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V79', 'V80', 'V81', 'V

### data description from a comment
> **ransaction table** “It contains money transfer and also other gifting goods  and service, like you booked a ticket for others, etc.”
 * **TransactionDT**: timedelta from a given reference datetime (not an actual timestamp)
 “TransactionDT first value is 86400, which corresponds to the number of seconds in a day (60 * 60 * 24 = 86400) so
  I think the unit is seconds. Using this, we know the data spans 6 months, as the maximum value is 15811131, which
 would correspond to day 183.”
 * **TransactionAMT**: transaction payment amount in USD
 “Some of the transaction amounts have three decimal places to the right of the decimal point. There seems to be a
  link to three decimal places and a blank addr1 and addr2 field. Is it possible that these are foreign transactions
  and that, for example, the 75.887 in row 12 is the result of multiplying a foreign currency amount by an exchange rate?”
 * **ProductCD**: product code, the product for each transaction
 “Product isn't necessary to be a real 'product' (like one item to be added to the shopping cart). It could be any kind of
  service.”
 * **card1 - card6**: payment card information, such as card type, card category, issue bank, country, etc.
 * **addr**: address
 “both addresses are for purchaser
 addr1 as billing region
 addr2 as billing country”
 * **dist**: distance
 "distances between (not limited) billing address, mailing address, zip code, IP address, phone area, etc.”
 P_ and (R_) emaildomain: purchaser and recipient email domain “ certain transactions don't need recipient, so
 Remaildomain is null.”
 * **C1-C14**: counting, such as how many addresses are found to be associated with the payment card, etc. The actual meaning
 is masked.
 “Can you please give more examples of counts in the variables C1-15? Would these be like counts of phone numbers, email
 addresses, names associated with the user? I can't think of 15.
 Your guess is good, plus like device, ipaddr, billingaddr, etc. Also these are for both purchaser and recipient, which
  doubles the number.”
 * **D1-D15**: timedelta, such as days between previous transaction, etc.
 * **M1-M9**: match, such as names on card and address, etc.
 * **Vxxx**: Vesta engineered rich features, including ranking, counting, and other entity relations.
 “For example, how many times the payment card associated with a IP and email or address appeared in 24 hours time range,
  etc.”
 * **Identity Table**
 Variables in this table are identity information – network connection information (IP, ISP, Proxy, etc) and digital
 signature (UA/browser/os/version, etc) associated with transactions.
 They're collected by Vesta’s fraud protection system and digital security partners.
 (The field names are masked and pairwise dictionary will not be provided for privacy protection and contract agreement)
 ***“id01 to id11** are numerical features for identity, which is collected by Vesta and security partners such as device
 rating, ip_domain rating, proxy rating, etc. Also it recorded behavioral fingerprint like account login times/failed to login times, how long an account stayed on the page, etc. All of these are not able to elaborate due to security partner T&C. I hope you could get basic meaning of these features, and by mentioning them as numerical/categorical, you won't deal with them inappropriately.”

 ## make selection of data
> current feature set is to big too clean up easily. cleanup data per feature and scale features later 

In [0]:
desiredFeatures = ['isFraud', 'TransactionAmt', 'TransactionDT', 'ProductCD', 'card1', 'card2',
                    'card3', 'card4', 'card5', 'card6', 'dist1', 'dist2', 'P_emaildomain', 'R_emaildomain',
                    'D1', 'D3', 'D5', 'D8', 'D9', 'D10',
                    'D11', 'D12', 'D13', 'D14', 'D15'
                    ]

def featureEngineering(data):
  data['D9'] = (data['TransactionDT']%(3600*24)/3600//1)/24.0

df_subset = df[desiredFeatures]
featureEngineering(df_subset)

df_subset.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,isFraud,TransactionAmt,TransactionDT,ProductCD,card1,card2,card3,card4,card5,card6,dist1,dist2,P_emaildomain,R_emaildomain,D1,D3,D5,D8,D9,D10,D11,D12,D13,D14,D15
0,0,68.5,86400,W,13926,NaN,150.0,discover,142.0,credit,19.0,NaN,NaN,NaN,14.0,13.0,NaN,NaN,0.0,13.0,13.0,NaN,NaN,NaN,0.0
1,0,29.0,86401,W,2755,404.0,150.0,mastercard,102.0,credit,NaN,NaN,gmail.com,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0
2,0,59.0,86469,W,4663,490.0,150.0,visa,166.0,debit,287.0,NaN,outlook.com,NaN,0.0,NaN,NaN,NaN,0.0,0.0,315.0,NaN,NaN,NaN,315.0
3,0,50.0,86499,W,18132,567.0,150.0,mastercard,117.0,debit,NaN,NaN,yahoo.com,NaN,112.0,0.0,0.0,NaN,0.0,84.0,NaN,NaN,NaN,NaN,111.0
4,0,50.0,86506,H,4497,514.0,150.0,mastercard,102.0,credit,NaN,NaN,gmail.com,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN


### discarded feature list:
* **addr1, addr2**  probably not a very good predictor on fraud. only if it has learned that one adress does or recieves it often but thats not what I want the system to learn. missing values is also a problem ~ 11%
* **d11** to many missing 

## understand data selection

### pre understanding data cleanup
the cleanup can be validated this way

using profiling to understand the data

In [0]:
import pandas_profiling as pandprof
df_subset.sample(frac=0.0001).profile_report(style={'full_width':True})

## feature cleanup
using the log of transactionAmt to describe the big numbers better


In [0]:
#df_cleaned = df_subset
#df_cleaned = df.copy(deep=True).sample(frac=0.01)# fraction for speedy testing

df_cleaned = df.copy(deep=True)

In [0]:
def AddNaNMask(data, featureName):
  newFeatureName = featureName + "_nan"
  data[newFeatureName] = data[featureName].isnull().astype(int)
  #print("\t AddNaNMask(" + featureName + ") done")

minNumberOfNa = len(df_cleaned)*0.001 # .1 percent
df_number_of_na = df_cleaned.isna().sum() 

featuresToApplyNanMasksTo = df_cleaned.loc[:,df_number_of_na > minNumberOfNa]
#featuresToApplyNanMasksTo = ['D10', 'D11', 'D12', 'D13', 'D15', 'D3', 'D5', 'D8', 'dist1', 'dist2']
print("applied na mask to: " + str(featuresToApplyNanMasksTo.columns.tolist()))

def applyNanMasks(data):
  for featureToNaNMask in tqdm(featuresToApplyNanMasksTo):
    AddNaNMask(data, featureToNaNMask)

def LogFeatures(data, featureName):
  # x + .01 to get rid of inf(a 0)
  data[featureName] = data[featureName].map(lambda x:np.log(x+.01))
  print("\t logFeature(" + featureName + ") done")

featuresToUseLogOn = []#['TransactionAmt','dist2','dist1','D1','D10','D11','D12','D15','D13', 'D14', 'D15', 'D3', 'D5', 'D8']

def useLog(data):
  for featureToLog in featuresToUseLogOn:
    LogFeatures(data, featureToLog )

applyNanMasks(df_cleaned)
#useLog(df_cleaned)
df_cleaned.head()

  0%|          | 11/590540 [00:00<1:33:17, 105.50it/s]

applied na mask to: ['card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'dist1', 'dist2', 'P_emaildomain', 'R_emaildomain', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V79', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V86', 'V87', 'V88', 'V89', 'V90', 'V91', 'V92', 'V93', 'V94', 'V138', 'V139', 'V140', 'V141', 'V142', 'V143', 'V144', 'V145', '

  0%|          | 339/590540 [00:03<1:46:18, 92.52it/s]


,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,id_01_nan,id_02_nan,id_03_nan,id_04_nan,id_05_nan,id_06_nan,id_07_nan,id_08_nan,id_09_nan,id_10_nan,id_11_nan,id_12_nan,id_13_nan,id_14_nan,id_15_nan,id_16_nan,id_17_nan,id_18_nan,id_19_nan,id_20_nan,id_21_nan,id_22_nan,id_23_nan,id_24_nan,id_25_nan,id_26_nan,id_27_nan,id_28_nan,id_29_nan,id_30_nan,id_31_nan,id_32_nan,id_33_nan,id_34_nan,id_35_nan,id_36_nan,id_37_nan,id_38_nan,DeviceType_nan,DeviceInfo_nan
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,330.0,87.0,287.0,NaN,outlook.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,87.0,NaN,NaN,yahoo.com,NaN,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,NaN,NaN,NaN,NaN,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,420.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,1,1,1,1,1,1,1,1,0,0,1,0,0,0,0,1,0,0,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# credit to 
# func that returns a dummified DataFrame of significant dummies in a given column
def dum_sign(dummy_col, threshold=0.1):

    # removes the bind
    dummy_col = dummy_col.copy()

    # what is the ratio of a dummy in whole column
    count = pd.value_counts(dummy_col) / len(dummy_col)

    # cond whether the ratios is higher than the threshold
    mask = dummy_col.isin(count[count > threshold].index)

    # replace the ones which ratio is lower than the threshold by a special name
    dummy_col[~mask] = "others"

    return pd.get_dummies(dummy_col, prefix=dummy_col.name)

def my_dummiefied_func(data):
  cols = data.columns
  num_cols = data._get_numeric_data().columns
  categorical_colums = list(set(cols) - set(num_cols))
  print("dummyfied: " + str(categorical_colums))
  for categorical_colum in tqdm(categorical_colums):
    dum_data = dum_sign(data[categorical_colum])
    for new_columns in dum_data.columns:
      data[new_columns] = dum_data[new_columns]
    data = data.drop(categorical_colum, axis=1)
  return data

In [0]:
#making dummies of categories. Na's are preserved becouse they could carry meaning
#df_cleaned = pd.get_dummies(df_cleaned, dummy_na= True)
#fill na with the mean of the data
#df_cleaned = df_cleaned.fillna(df_cleaned.mean())

df_cleaned = my_dummiefied_func(df_cleaned)
df_cleaned = df_cleaned.fillna(-1.0)
df_cleaned.head()

  0%|          | 0/31 [00:00<?, ?it/s]

dummyfied: ['id_23', 'card4', 'M4', 'id_37', 'id_15', 'M9', 'id_31', 'M5', 'M1', 'M6', 'id_36', 'id_27', 'M8', 'DeviceType', 'M7', 'M3', 'id_30', 'id_35', 'id_33', 'id_12', 'DeviceInfo', 'P_emaildomain', 'id_16', 'R_emaildomain', 'ProductCD', 'card6', 'id_38', 'M2', 'id_29', 'id_28', 'id_34']


100%|██████████| 31/31 [00:42<00:00,  1.32s/it]


,TransactionID,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,dist1,dist2,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,...,DeviceType_others,M7_F,M7_others,M3_F,M3_T,M3_others,id_30_others,id_35_F,id_35_T,id_35_others,id_33_others,id_12_NotFound,id_12_others,DeviceInfo_others,P_emaildomain_gmail.com,P_emaildomain_others,P_emaildomain_yahoo.com,id_16_Found,id_16_NotFound,id_16_others,R_emaildomain_others,ProductCD_C,ProductCD_W,ProductCD_others,card6_credit,card6_debit,card6_others,id_38_F,id_38_T,id_38_others,M2_T,M2_others,id_29_Found,id_29_NotFound,id_29_others,id_28_Found,id_28_New,id_28_others,id_34_match_status:2,id_34_others
0,2987000,0,86400,68.5,13926,-1.0,150.0,142.0,315.0,87.0,19.0,-1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,-1.0,13.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,13.0,13.0,-1.0,-1.0,-1.0,...,1,0,1,0,1,0,1,0,0,1,1,0,1,1,0,1,0,0,0,1,1,0,1,0,1,0,0,0,0,1,1,0,0,0,1,0,0,1,0,1
1,2987001,0,86401,29.0,2755,404.0,150.0,102.0,325.0,87.0,-1.0,-1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,...,1,0,1,0,0,1,1,0,0,1,1,0,1,1,1,0,0,0,0,1,1,0,1,0,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,1
2,2987002,0,86469,59.0,4663,490.0,150.0,166.0,330.0,87.0,287.0,-1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,315.0,-1.0,-1.0,-1.0,...,1,1,0,0,1,0,1,0,0,1,1,0,1,1,0,1,0,0,0,1,1,0,1,0,0,1,0,0,0,1,1,0,0,0,1,0,0,1,0,1
3,2987003,0,86499,50.0,18132,567.0,150.0,117.0,476.0,87.0,-1.0,-1.0,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,-1.0,-1.0,-1.0,-1.0,84.0,-1.0,-1.0,-1.0,-1.0,...,1,0,1,0,0,1,1,0,0,1,1,0,1,1,0,0,1,0,0,1,1,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,1
4,2987004,0,86506,50.0,4497,514.0,150.0,102.0,420.0,87.0,-1.0,-1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1,0,1,0,0,1,1,0,1,0,1,1,0,1,1,0,0,0,1,0,1,0,0,1,1,0,0,0,1,0,0,1,0,1,0,0,1,0,1,0


### non functional ram reduction script

In [0]:
# credit to @guiferviz for the memory reduction  
def memory_usage_mb(df, *args, **kwargs):
    """Dataframe memory usage in MB. """
    return df.memory_usage(*args, **kwargs).sum() / 1024**2

def reduce_memory_usage(df, deep=True, verbose=True):
    # All types that we want to change for "lighter" ones.
    # int8 and float16 are not include because we cannot reduce
    # those data types.
    # float32 is not include because float16 has too low precision.
    numeric2reduce = ["int16", "int32", "int64", "float64"]
    start_mem = 0
    if verbose:
        start_mem = memory_usage_mb(df, deep=deep)

    for col, col_type in tqdm(df.dtypes.iteritems()):
        best_type = None
        if col_type in numeric2reduce:
            downcast = "integer" if "int" in str(col_type) else "float"
            df[col] = pd.to_numeric(df[col], downcast=downcast)
            best_type = df[col].dtype.name
        # Log the conversion performed.
        if verbose and best_type is not None and best_type != str(col_type):
            print(f"Column '{col}' converted from {col_type} to {best_type}")

    if verbose:
        end_mem = memory_usage_mb(df, deep=deep)
        diff_mem = start_mem - end_mem
        percent_mem = 100 * diff_mem / start_mem
        print(f"Memory usage decreased from"
              f" {start_mem:.2f}MB to {end_mem:.2f}MB"
              f" ({diff_mem:.2f}MB, {percent_mem:.2f}% reduction)")
        
    return df

In [0]:
df_cleaned = reduce_memory_usage(df_cleaned, verbose=False)

812it [03:56,  3.43it/s]


In [0]:
def memory_usage_mb(df, *args, **kwargs):
    """Dataframe memory usage in MB. """
    return df.memory_usage(*args, **kwargs).sum() / 1024**2
memory_usage_mb(df_cleaned, deep=True)

1139.8820495605469

In [0]:
import gc
gc.collect()

0

## feature sellection

# Train model


In [0]:
from sklearn.model_selection import train_test_split
X = df_cleaned.drop(labels='isFraud',axis=1)
Y = df_cleaned['isFraud']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=47, stratify=Y)
print([X_train.shape, y_train.shape], [X_test.shape, y_test.shape])

[(395661, 811), (395661,)] [(194879, 811), (194879,)]


In [0]:
from sklearn.metrics import roc_curve, classification_report, confusion_matrix, auc

def Score(truth, predict):
  fpr, tpr, thresholds = roc_curve(truth, predict)
  print ("auc: "+ str(auc(fpr, tpr)))
  tn, fp, fn, tp = confusion_matrix(truth,predict).ravel()
  print ("tp: "+ str(tp)+ "\tfp v : "+ str(fp)+\
         "\nfn ^ : "+ str(fn) + "\ttn:"+str(tn))
  print (classification_report(truth, predict,labels=[0,1]))

##random forrest

In [0]:
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier(n_estimators=100,max_depth=42, verbose=True, n_jobs=-1,random_state=True)
RFC.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  3.1min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=42, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=True,
                       verbose=True, warm_start=False)

In [0]:
Y_hat_test = RFC.predict(X_test)
Score(y_test, Y_hat_test)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    1.9s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    4.4s finished


auc: 0.7139759615460345
tp: 2926	fp v : 215
fn ^ : 3893	tn:187845
              precision    recall  f1-score   support

           0       0.98      1.00      0.99    188060
           1       0.93      0.43      0.59      6819

    accuracy                           0.98    194879
   macro avg       0.96      0.71      0.79    194879
weighted avg       0.98      0.98      0.98    194879



In [0]:
y_hat_train = RFC.predict(X_train)
Score(y_train, y_hat_train)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    4.1s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    9.5s finished


auc: 0.9403712799768853
tp: 12193	fp v : 0
fn ^ : 1651	tn:381817
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    381817
           1       1.00      0.88      0.94     13844

    accuracy                           1.00    395661
   macro avg       1.00      0.94      0.97    395661
weighted avg       1.00      1.00      1.00    395661



In [0]:
featureImportance = RFC.feature_importances_
columns = X_train.columns
featureImportance = pd.DataFrame(np.stack((columns, featureImportance), axis=-1))
featureImportance.sort_values(1,ascending=False)

,0,1
0,TransactionID,0.0198138
1,TransactionDT,0.0197727
2,TransactionAmt,0.0185931
3,card1,0.0172869
4,card2,0.0160554
23,C13,0.0160062
11,C1,0.0157275
297,V258,0.0138186
7,addr1,0.0130114
24,C14,0.0113731


#Predict
## aqire data

In [0]:
df_validate = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Datasets/fraud_detection/test_transaction.csv')
df_validate_identity = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Datasets/fraud_detection/test_identity.csv', na_values=missing_values)
df = pd.merge(df_validate, df_validate_identity, on='TransactionID', how='left')
df_validate.shape

(506691, 393)

## clean aqired data & make features the same

In [0]:
featureEngineering(df_validate)# select and make new features based on training set function

useLog(df_validate) # use logarithmic scale for features
df_validate = pd.get_dummies(df_validate, dummy_na= True) #

_ , df_validata_cleaned = X_train.align(df_validate, join='left', axis=1)

df_validata_cleaned = df_validata_cleaned.fillna(df_cleaned.mean())

print (X_train.shape)
print (df_validata_cleaned.shape)

(295271, 146)
(506691, 146)



## make prediction

In [0]:
 RFC.predict(df_validata_cleaned)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.3s finished


array([1, 1, 1, ..., 1, 1, 0])